# Импорт модулей

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost

import json
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import log_loss

# Загрузка данных

Тренировочные данные состоят из полей 'text_id', 'text' и 11 полей с таргетами. Не стоит забывать, что может быть больше одной болезни для каждого случая.

Тестовые же данные содержат поля 'text_id' и 'text'.

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Agrocode Hack/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Agrocode Hack/test.csv')

In [ ]:
train['text'].head(10)

0    Корова, видимо вставая, раздавила себе сосок. ...
1    Корове 8 лет! Месяц назад промеж четвертей вым...
2    Молоко течёт само у коровы. Что делать, если у...
3    У нетели болячки на вымени.\nЗдравствуйте. Нет...
4    У меня первотелка, на днях отёл, у неё левый п...
5    Чем пропоить телёнка от поноса?\nПодскажите по...
6    У лейкозной коровы язвочки на сосках, что это?...
7    Здравствуйте у меня проблема. Бык-1,2 года вне...
8    Прошу совета в лечении язвочек на вымени у кор...
9    Можно ли смешать сыворотку от паратифа с антиб...
Name: text, dtype: object

In [ ]:
test.head(2)

,text_id,text
0,294,Понос у месячных телят. Подскажите методы и сп...
1,295,"Понос у телят, чем лечить? \nЧем можно вылечит..."


# EDA

In [ ]:
from gensim.models import Word2Vec

In [ ]:
w2v_model = Word2Vec(min_count=2, window=2, size=50, negative=10, alpha=0.03, min_alpha=0.0007, sample=6e-5, sg=1)
# min_count — игнорировать все слова с частотой встречаемости меньше, чем это значение. 
# windоw — размер контекстного окна, о котором говорили тут, обозначает диапазон контекста.
# size — размер векторного представления слова (word embedding).
# negative — сколько неконтекстных слов учитывать в обучении, используя negative sampling, о нем также упоминалось здесь.
# alpha — начальный learning_rate, используемый в алгоритме обратного распространения ошибки (Backpropogation).
# min_alpha — минимальное значение learning_rate, на которое может опуститься в процессе обучения.
# sg — если 1, то используется реализация Skip-gram; если 0, то CBOW. О реализациях также говорили тут.

In [ ]:
# загужаем датасет
w2v_model.build_vocab(train['text'])

In [ ]:
# обучаем модель
w2v_model.train(train['text'], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(149146, 2767260)

In [ ]:
w2v_model.wv.most_similar(positive=['Понос'])

KeyError: ignored

# Базовая модель

В качестве базового решения используется CatBoostClassifier, поддерживающий текстовые фичи. Помимо этого задача является мультилейбл классификацией, поэтому модель обернута в OneVsRestClassifier.

**Делим данные на тренировочную и валидационную выборку**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(pd.DataFrame(train['text']), train[train.columns[2:]], test_size=0.2)

**Обучаем модель**

In [ ]:
model = OneVsRestClassifier(estimator=CatBoostClassifier(iterations = 100, text_features=['text'], 
                                                         verbose = 50, allow_writing_files=False))
model.fit(X_train, y_train)

Learning rate set to 0.045854
0:	learn: 0.6750223	total: 90.4ms	remaining: 8.95s
50:	learn: 0.3992793	total: 1.85s	remaining: 1.78s
99:	learn: 0.3346489	total: 3.54s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6736297	total: 35.3ms	remaining: 3.5s
50:	learn: 0.3749512	total: 1.75s	remaining: 1.68s
99:	learn: 0.3172577	total: 3.4s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6745299	total: 34ms	remaining: 3.37s
50:	learn: 0.3882340	total: 1.78s	remaining: 1.71s
99:	learn: 0.3269351	total: 3.48s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6714341	total: 38.2ms	remaining: 3.78s
50:	learn: 0.3663993	total: 1.74s	remaining: 1.68s
99:	learn: 0.3109305	total: 3.42s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6741982	total: 35.9ms	remaining: 3.56s
50:	learn: 0.3477512	total: 1.76s	remaining: 1.7s
99:	learn: 0.2814248	total: 3.43s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6723350	total: 33.7ms	remaining: 3.33s
50:	learn: 0.347180

OneVsRestClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7f6c6a26ae10>,
                    n_jobs=None)

# Считаем метрику

В качестве "gt" (ground truth) функция принимает на вход датафрейм/массив из 10 столбцов (все классы, кроме "другое"). Предсказанные значения – "pr", также должны быть либо в виде датафрейма, либо в виде массива.

In [ ]:
def log_loss_score(gt, pr):
    
    log_loss_ = 0
    
    gt = np.array(gt)
    
    for i in range(10):
        log_loss_ += log_loss(gt[:, i], pr[:, i])
        
    return log_loss_ / 10

In [ ]:
log_loss_score(y_val, model.predict_proba(X_val))

0.35142202402040995

# Создание файла отправки

В файле с ответами должны быть вероятности для каждого класса. Сумма вероятностей в каждой строке может быть больше 1.

In [ ]:
submission_columns = ['text_id'] + list(train.columns[2:-1])
submission = pd.concat([test['text_id'], pd.DataFrame(model.predict_proba(pd.DataFrame(test['text']))[:, :10])], axis=1)
submission.columns = submission_columns

In [ ]:
submission.head()

,text_id,эймериоз,абсцесс,диспепсия молодняка,остертагиоз,мастит,инфекционный ринотрахеит,отёк вымени,тенденит,сибирская язва,лишай
0,294,0.062719,0.113136,0.072705,0.074253,0.103621,0.075186,0.066241,0.057393,0.104223,0.061639
1,295,0.567817,0.143268,0.392205,0.375057,0.211739,0.133588,0.136544,0.406265,0.173632,0.088913
2,296,0.198515,0.120845,0.369088,0.180145,0.233022,0.078119,0.065792,0.311344,0.104504,0.063568
3,297,0.065979,0.148187,0.077228,0.087859,0.083386,0.069549,0.060314,0.062278,0.103982,0.068143
4,298,0.068813,0.138642,0.084449,0.083486,0.188230,0.067497,0.089238,0.067269,0.115569,0.089609


In [ ]:
submission_json = {str(k): {"span": list(), "label": list(v.values())} \
                   for k,v in submission.set_index('text_id').to_dict('index').items()}

submission_json['294']

{'span': [],
 'label': [0.0627194363685613,
  0.11313570529902758,
  0.07270482269352871,
  0.0742534689467756,
  0.10362104310746847,
  0.07518588621133018,
  0.06624119018743685,
  0.05739265274755671,
  0.10422273080497868,
  0.061638782036372346]}

In [ ]:
with open('sample_submission.json', 'w') as final_submit:
    json.dump(submission_json, final_submit, indent=4)